In [146]:
# Library
import pandas as pd
import numpy as np
import calendar
import copy
import datetime
from tqdm import tqdm

In [147]:
df = pd.read_csv('daily_transactions.csv', sep=',')
print(df.shape)
df.head(5)

(1210998, 3)


,_id,event_date,usertype
0,By1+rEy20nW/sgRehb+RSZe9VI8=,2021-02-10,A
1,x7lxSW+y6Kymzxf1yrriMXDNZWQ=,2021-02-10,A
2,BQ7feJAnc6ntg4PaOXShHLrd3xQ=,2021-02-24,A
3,xmc9CsW3Q7K9HK5+pczuRvKUCBk=,2020-12-08,A
4,Dt0y6YCKnC/mD9Sm1SkNkwsrgzY=,2020-06-17,A


In [148]:
# Convert column from obj to datetime
df['event_date'] = pd.to_datetime(df['event_date'])

# Sort dataframe
df = df[df['event_date'] <= '2021-04-30']
df = df.sort_values(by=['_id', 'event_date'])

In [149]:
common_ids = list(df['_id'].unique())
group = df.groupby(['_id'])

# Check type
def is_unique(i):
    a = i.to_numpy()
    return (a[0] == a).all()

for c in tqdm(common_ids):
    g = group.get_group(c)

    if len(g) > 1:
        result = is_unique(g['usertype'])
        if result == False:
            df.drop(g.index)

100%|██████████| 432458/432458 [00:49<00:00, 8699.44it/s]


In [150]:
# Store result
months = list(calendar.month_name[1:])
years = [2020, 2021]
newUser = {}

# Create dict to store new user
for y in years:
    for m in months:
        newUser["{}-{}".format(m, y)] = 0

# Copy
dropOff = copy.deepcopy(newUser)
returnUser = copy.deepcopy(newUser)

In [151]:
# Convert column from obj to datetime
df['event_date'] = pd.to_datetime(df['event_date'])

# Sort dataframe and drop duplicate
df_ft = df.drop_duplicates(subset=['_id'], keep='first')
df_ft = df_ft.sort_values(by='event_date')
df_ft['first_tran'] = True
df_ft_merge = df_ft
df_ft = df_ft[df_ft['event_date'] >= '2020-04-01']

# Count new user for each months
for _, row in tqdm(df_ft.iterrows()):
    newUser["{}-{}".format(calendar.month_name[row['event_date'].month], row['event_date'].year)] += 1

# New users
newUser = {k:v for k,v in newUser.items() if v != 0}
newUser

311386it [00:15, 20456.32it/s]


{'April-2020': 15117,
 'May-2020': 20295,
 'June-2020': 22949,
 'July-2020': 30152,
 'August-2020': 28019,
 'September-2020': 21693,
 'October-2020': 22854,
 'November-2020': 18875,
 'December-2020': 19071,
 'January-2021': 23389,
 'February-2021': 22676,
 'March-2021': 31369,
 'April-2021': 34927}

In [152]:
# Sort dataframe and drop duplicate
df_lt_churn = df.drop_duplicates(subset=['_id'], keep='last')
df_lt_churn = df_lt_churn.sort_values(by='event_date')
df_lt_merge = df_lt_churn

# Churn periods
churn = {'A': 360, 'B': 360, 'C': 120, 'D': 260}

# Add the churn values to all of the row
for idx, row in tqdm(df_lt_churn.iterrows()):
    df_lt_churn.at[idx, 'event_date'] = row['event_date'] + datetime.timedelta(days=churn[row['usertype']])

# Filter
df_lt_churn = df_lt_churn[(df_lt_churn['event_date'] >= '2020-04-01') & (df_lt_churn['event_date'] <= '2021-04-30')]

# Count drop off user for each months
for _, row in tqdm(df_lt_churn.iterrows()):
    dropOff["{}-{}".format(calendar.month_name[row['event_date'].month], row['event_date'].year)] += 1

# Old users
dropOff

432458it [00:38, 11240.85it/s]
131718it [00:08, 15602.71it/s]


{'January-2020': 0,
 'February-2020': 0,
 'March-2020': 0,
 'April-2020': 675,
 'May-2020': 7646,
 'June-2020': 10327,
 'July-2020': 11909,
 'August-2020': 10768,
 'September-2020': 9886,
 'October-2020': 9867,
 'November-2020': 9457,
 'December-2020': 9058,
 'January-2021': 12710,
 'February-2021': 12347,
 'March-2021': 12049,
 'April-2021': 15019,
 'May-2021': 0,
 'June-2021': 0,
 'July-2021': 0,
 'August-2021': 0,
 'September-2021': 0,
 'October-2021': 0,
 'November-2021': 0,
 'December-2021': 0}

In [153]:
# Add columns
df_lt_merge['last_tran'] = True

# Filter
df = df[df['event_date'] <= '2021-04-30']

# Merge
df_merge = pd.merge(df_ft_merge, df_lt_merge, how="outer", on=['_id', 'event_date', 'usertype'])
df_merge = pd.merge(df_merge, df, how="outer", on=['_id', 'event_date', 'usertype'])
df_merge = df_merge.replace(np.nan, False)

# Remove value with single transaction (true & true)
df_merge = df_merge[(df_merge['first_tran'] != True) | (df_merge['last_tran'] != True)]
# df_merge = df_merge[(df_merge['first_tran'] != True)]
df_merge

,_id,event_date,usertype,first_tran,last_tran
0,WfVl2UFHQXU3iIOZyyuyjRccUgo=,2019-05-13,B,True,False
1,7mipoXGVQZq6/rW0j2aKY+AReDk=,2019-05-13,A,True,False
2,cmicF5fh0PE4a32wIERHoil1IdM=,2019-05-13,A,True,False
3,6kkeH6lJflikhoAIspU2LOwrJaE=,2019-05-13,B,True,False
4,7Iw2SCuQAFS4RFzdWHDbfo6QMWA=,2019-05-13,B,True,False
...,...,...,...,...,...
1633268,zzyMFNFuAwqAN1FMkTmCD7gs1fU=,2021-03-01,A,False,False
1633269,zzyMFNFuAwqAN1FMkTmCD7gs1fU=,2021-03-02,A,False,False
1633270,zzyMFNFuAwqAN1FMkTmCD7gs1fU=,2021-03-03,A,False,False
1633271,zzyMFNFuAwqAN1FMkTmCD7gs1fU=,2021-03-04,A,False,False


In [154]:
all_ids = df_merge['_id'].unique()
group = df_merge.groupby(['_id'])

# Check type
def count_returned(i):
    a = i.to_numpy()
    churn_period = a[0][1] + datetime.timedelta(days=churn[a[0][2]])
    
    for x in a:
        if x[1] > churn_period:
            if (x[1].year in years) and (churn_period.year in years):
                dropOff["{}-{}".format(calendar.month_name[churn_period.month], churn_period.year)] += 1
                returnUser["{}-{}".format(calendar.month_name[x[1].month], x[1].year)] += 1

            if x[4] != True:
                churn_period = x[1] + datetime.timedelta(days=churn[x[2]])
    
for c in tqdm(all_ids):
    g = group.get_group(c)

    if len(g) > 1:
        count_returned(g)

returnUser

100%|██████████| 432458/432458 [01:33<00:00, 4627.87it/s]


{'January-2020': 199,
 'February-2020': 318,
 'March-2020': 385,
 'April-2020': 353,
 'May-2020': 3126,
 'June-2020': 4544,
 'July-2020': 5215,
 'August-2020': 4899,
 'September-2020': 4715,
 'October-2020': 4949,
 'November-2020': 5069,
 'December-2020': 5008,
 'January-2021': 7451,
 'February-2021': 7268,
 'March-2021': 7277,
 'April-2021': 8838,
 'May-2021': 9950,
 'June-2021': 10554,
 'July-2021': 14728,
 'August-2021': 15330,
 'September-2021': 10338,
 'October-2021': 11402,
 'November-2021': 9626,
 'December-2021': 9766}

In [155]:
# Filter to the only desired target
def filter_result(x, y):
    for i in x:
        del y[i]

del_list = ['January-2020', 'February-2020', 'March-2020', 'May-2021', 'June-2021', 'July-2021', 'August-2021', 'September-2021', 'October-2021', 'November-2021', 'December-2021']
filter_result(del_list, returnUser)
filter_result(del_list, dropOff)

returnUser

{'April-2020': 353,
 'May-2020': 3126,
 'June-2020': 4544,
 'July-2020': 5215,
 'August-2020': 4899,
 'September-2020': 4715,
 'October-2020': 4949,
 'November-2020': 5069,
 'December-2020': 5008,
 'January-2021': 7451,
 'February-2021': 7268,
 'March-2021': 7277,
 'April-2021': 8838}

In [156]:
dropOff

{'April-2020': 1331,
 'May-2020': 14094,
 'June-2020': 16705,
 'July-2020': 18559,
 'August-2020': 16470,
 'September-2020': 15395,
 'October-2020': 15487,
 'November-2020': 14897,
 'December-2020': 14479,
 'January-2021': 20325,
 'February-2021': 19484,
 'March-2021': 18434,
 'April-2021': 23075}

In [167]:
month_list = []

# Create month list
for y in years:
    for i, m in enumerate(months):
        if y == 2020:
            if i > 2:
                month_list.append('{} {}'.format(m, y))
        if y == 2021:
            if i < 4:
                month_list.append('{} {}'.format(m, y))

df_final = pd.DataFrame({'month': month_list, 'dropoff_users': pd.Series(dropOff), 'returning_user': pd.Series(returnUser), 'new_users': pd.Series(newUser)})
df_final

,month,dropoff_users,returning_user,new_users
April-2020,April 2020,1331,353,15117
May-2020,May 2020,14094,3126,20295
June-2020,June 2020,16705,4544,22949
July-2020,July 2020,18559,5215,30152
August-2020,August 2020,16470,4899,28019
September-2020,September 2020,15395,4715,21693
October-2020,October 2020,15487,4949,22854
November-2020,November 2020,14897,5069,18875
December-2020,December 2020,14479,5008,19071
January-2021,January 2021,20325,7451,23389


In [168]:
df_final.to_csv('result.csv', index=False)